## FM

FM (Factorization Machines), 因式分解机。

TLDR;

1. 基础知识点：SVD

2. FM 解决的问题

    - 特征稀疏 （SVM Failed)
    
    - 可以线性计算 (SVM 不能）

3. 应用
    - 回归 
    - 分类
    - 推荐（Ranking)

FM 还可替换其它学习算法，如 SVD++, SVM 等。

SVM 在稀疏矩阵上不能更好的学习到参数来分隔超平面。

### 背景

在现实的问题中，对于 categorical 的特征，我们在学习时，通常都会使用 one-hot encodig 进行编码。这样生成的特征就会出现很大稀疏，SVM 在第处理这种问题的时候就很无力了。

以下内容来自美团技术上的介绍--[深入FFM原理与实践](https://tech.meituan.com/2016/03/03/deep-understanding-of-ffm-principles-and-practices.html)。

-----

FM 由 Steffen Rendle 于 2010 年最早提出的，旨在解决稀疏数据下的特征组合问题。下面以一个示例引入 FM 模型。

假设一个广告分类的问题，根据用户和广告位相关的特征，预测用户是否点击了广告。源数据如下：

| Clicked | Country | Day      | Ad_type |
|---------|---------|----------|---------|
| 1       | USA     | 3/3/15   | Moive   |
| o       | China   | 1/7/14   | Game    |
| 1       | China   | 3/3/15   | Game    |

是否点击 `Clicked` 是 `label`, 特征是：`Country, Day, Ad_type`。这三种特征都是 categorical 类型的，所以需要经过独热编码 (one-hot encoding) 转换成数值特征。 即

| Clicked | Country=USA |Country=USA | Day=3/3/15 | Day=1/7/14 |Ad_type=Movie | Ad_type=Game |
|---------|-------------|------------|------------|------------|--------------|--------------|
| 1       | 1           | 0          | 1          | 0          | 1            | 0            |
| 0       | 0           | 1          | 0          | 1          | 0            | 1            |
| 1       | 0           | 1          | 1          | 0          | 0            | 1            |


由上表可以看出，经过 One-hot 编码之后，大部分样本数据特征是比较稀疏 的。经过 One-hot 编码后，每个样本有 7 个特征，但平均仅有 3 个特征的取值是非零的。实际上，这种情况并不是此例独有的，在真实应用的场景中这种情况也是普遍存在。

例如，在 CTR (点击率，click-through reate)/CVR（转化率，conversion rate) 预测时，用户的性别、职业、教育水平、品类偏好、商品的品类等，经过 One-hot 编码后都会导致样本数据的稀疏性。特别是商品品类这种特征，如商品品类有 500 个级别，那么采用 One-hot 编码后会生成 500 个特征，但每个样本的这 500 个特征，有且仅有一个有效的 （非零）。由此可见，数据稀疏性是实际问题中不避免的挑战。

One-hot 编码的一个特点就是导致特征空间变大，特征空间剧增。

通过大量的样本数据可以发现，经过对特征进行关联之后，与 label 之间的相关性就会提高。例如，『USA』与 『Thansgiving』，『China』与 『Chinese New Year』这样的关联特征，对用户的点击有着正向的影响。这种关联特征与 label 的正向相关性在实际问题中是普遍存在的，如『化妆品』类商品与『女』性，『球类运动配件』的商品与『男』性，『电影票』的商品与『电影』品类偏好等。因此，引入两个特征的组合是非常有意义的。

**多项式模型**是包含特征组合的最直观的模型（SVM 通过不同的核也可以对特征进行组合）。模型表达式如下：

$$
y(\mathbf{x}) = w_0 + \sum_{i = 1}^{n} w_i x_i + \sum_{i = 1}^{n}\sum_{j = i + 1}^{j = n} w_{ij} x_i x_j  \tag{1}
$$

其中，n 代表样本的特征数量，$x_i$ 表示第 $i$ 个特征的值，$w_0, w_i, w_{ij}$ 是模型参数。

从公式 (1) 中可以看出，组合特征参数 $w_{ij}$ 的个数共有 $\frac{n(n - 1)}{2}$ 个，任意两个参数都是独立的。然而当数据稀疏时，二次项参数参数的训练很困难。其原因是，每个 $w_{ij}$ 的训练都需要大量 $x_i$ 和 $x_j$ 都非零的样本；但这们的样本是非常少的。最终训练出来的模型性能不会很好。

**那么，如何解决二次项参数的训练问题那？**

SVD 矩阵分解提供了一种解决思路。在 Model-based 的协同过滤中，一个 rating 矩阵可以分解为 user 矩阵和 item 矩阵，每一个 user 和 item 都可以采用一个隐向量表示。（SVD 在 LSI, LSA 应用原理）。




<img src="https://awps-assets.meituan.net/mit-x/blog-images-bundle-2016/1a91e67b.png" width="520px">


比如在上图中，第一行第一列都是一个隐向量。

对于二次项参数 $w_{ij}$ 组成的对称矩阵 $W$,

$$
W = 
 \begin{pmatrix}
  w_{1,1} & w_{1,2} & \cdots & w_{1,n} \\
  w_{2,1} & w_{2,2} & \cdots & w_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  w_{n,1} & w_{n,2} & \cdots & w_{n,n} 
 \end{pmatrix}
$$

其可以分解为 $W = V^T V$, $V$ 的第 $j$ 列即是第 $j$ 维特征的隐向量。这时每个参数都可以表示为 $ w_{ij} = <\mathbf{v_i}, \mathbf{v_j}>$, 这就是 FM 模型的核心思想。

因此，FM 的模型方程为 (d = 2, 仅讨论 2-way FM)

$$
y(\mathbf{x}) = w_0 + \sum_{i = 1}^{n} w_i x_i + \sum_{i = 1}^{n}\sum_{j = i + 1}^{j = n} <\mathbf{v}_i, \mathbf{v}_j> x_i x_j  \tag{2}
$$


其中，$\mathbf{v}_i \in \mathbb{R}^k, (k \ll n)$ 是第 $i$ 维特征隐向量。

公式（2）是一个通用的拟合方程，可以采用不同的损失函数来解决回归，分类等问题。如：使用 MSE 解决回归问题，使用 Hinge / Cross-entropy 解决分类问题。

直观上，FM 的复杂度是 $O(kn^2)$, 但是通过对称矩阵的化简可以证明其算法复杂度为 $O(nk)$。下面是推导过程,根据对称矩阵的 **上三角和 = （矩阵和 - 对称线和）/ 2** 来的推导。


\begin{align}
\sum_{i = 1}^{n} \sum_{j = i + 1}^{n} <\mathbf{v}_i, \mathbf{v}_j> x_i x_j 
&= \frac{1}{2} \sum_{i = 1}^{n} \sum_{j = 1}^{n} <\mathbf{v}_i, \mathbf{v}_j> x_i x_j - \frac{1}{2} \sum_{i = 1}^{n} <\mathbf{v}_i, \mathbf{v}_i> x_i x_i  \\
&= \frac{1}{2} \bigg( \sum_{i = 1}^{n} \sum_{j = 1}^{n} \sum_{f = 1}^{k} v_{i,f}v_{j,f}x_i x_j - \sum_{i = 1}^{n} \sum_{f = 1}^{k} v_{i,f}v_{i,f}x_i x_i \bigg)  \\
&= \frac{1}{2} \sum_{f = 1}^{k} \bigg( \big( \sum_{i = 1}^{n}v_{i, f}x_i \big) \big( \sum_{j = 1}^{n}v_{j, f}x_j \big) - \sum_{i = 1}^{n} v_{i,f}^2 x_i^2 \bigg) \\
&= \frac{1}{2} \sum_{f = 1}^{k} \bigg(  \big( \sum_{i = 1}^{n}v_{i, f}x_i \big)^2  - \sum_{i = 1}^{n} v_{i,f}^2 x_i^2 \bigg) 
\end{align}

通过 (2) 式，我可以获得各个参数的梯度如：

$$
\frac{\partial}{\partial \theta_j} y(\mathbf{x}) =\begin{cases}
               1, & \text{if} \space \theta \space \text{is} \space w_0 \\
               x_i, & \text{if} \space \theta \space \text{is} \space w_i \\
               x_i\sum_{j = 1}^{n}v_{j,f}x_j - v_{i,f}x_i^2, & \text{if} \space \theta \space \text{is} \space v_{i,f} \\
            \end{cases}
            \tag{4}
$$


基中，$v_{j, f}$ 是隐向量 $\mathbf{v}_j$ 的第 $f$ 个元素。由于 $\sum_{j = 1}^{n}v_{j, f}x_{j}$ 与 $i$ 无关，所以在迭代过程中需要计算一次就可以了。已知 $\sum_{j = 1}^{n}v_{j, f}x_{j}$ 与 $i$ 后，则每个参数梯度更新的复杂度为 $O(1)$, 模型一共有 $nk + n + 1$ 个参数，因此 FM 参数训练的复杂度也是 $O(kn)$。

综上可知，FM 可以在线性时间训练和预测，是一种非常高效的模型。

FM 经常使用 $L2$ 正则化。

### References

1. [Rendle S. Factorization machines[C] 2010 IEEE International Conference on Data Mining. IEEE, 2010: 995-1000.](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)
2. [深入FFM原理与实践 -- 美团](https://tech.meituan.com/2016/03/03/deep-understanding-of-ffm-principles-and-practices.html)
3. [Factorization Machines Slides](http://www.cs.cmu.edu/~wcohen/10-605/2015-guest-lecture/FM.pdf)
4. [LibFM 源代码](http://www.libfm.org/)
5. [CTR Prediction: From Linear Models to Field-aware Factorization Machines](https://ailab.criteo.com/ctr-prediction-linear-model-field-aware-factorization-machines/)